# Pandas

The Numpy library is excellent for numerical computations, but it lacks support to handle missing data or non-omogeneous arrays. The **Pandas** library is based on Numpy and extends the Numpy functionality, and is currently one of the most widely used tools for data manipulation, providing high-performance, easy-to-use data structures and advanced data analysis tools.

In particular Pandas features:

* A fast and efficient `Series` and `DataFrame` objects for data manipulation with integrated indexing;
* Tools for reading and writing data between in-memory data structures and different formats (CSV, Excel, SQL, HDF5);
* Smart data alignment and integrated handling of missing data;
* Time series-functionalities;
* Convenient label-based slicing, fancy indexing, and subsetting of large data sets;
* Hierarchical axis indexing provides an intuitive way of working with high-dimensional data in a lower-dimensional data structure;
* Aggregating and transforming data with a powerful "group-by" engine; 
* High performance merging and joining of data sets;
* Highly optimized for performance, with critical code paths written in Cython or C.

In [1]:
import pandas as pd # standard naming convention
import numpy as np

## Series

Pandas Series represent an extension of the Numpy 1D arrays. The content of a Series is equivalent to a Numpy array, and in addition the axis  is labeled. Labels don't need to be unique, but must be of a *hashable* type.

Since the content is of type `ndarray`, the content has to be *omogeneous*. However, there is the possibility to store heterogeneous data, but the content in this case would be of type `object`.

One of the most important examples are the **time series**, which are used to keep track of the time evolution of a certain quantity.

Link to the official Pandas Series [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html).

In [7]:
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

# Calling the Series constructor
# Constructor requires the data, and optionally the indices and data type
sr = pd.Series(np.arange(10)*0.5, index=letters[:10], dtype=float)
print("series:\n", sr, '\n')
print("series type:\n", type(sr), '\n')
print("indices:\n", sr.index, '\n')
print("values:", sr.values, type(sr.values), '\n') # values of the Series are actually a numpy array
print("type:\n", sr.dtype, '\n')

series:
 a    0.0
b    0.5
c    1.0
d    1.5
e    2.0
f    2.5
g    3.0
h    3.5
i    4.0
j    4.5
dtype: float64 

series type:
 <class 'pandas.core.series.Series'> 

indices:
 Index(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j'], dtype='object') 

values: [0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5] <class 'numpy.ndarray'> 

type:
 float64 



In [3]:
print("element by index:", sr['f'], '\n') # Accessing elements like arrays
print("element by attribute:", sr.f, '\n') # Accessing elements like attributes - not recommended

# selecting a subset of the series
subsr = sr[['d', 'f', 'h']] # note the double square brackets
print("series subset:\n", subsr, type(subsr), '\n') # Multiple indexing returns another series

element by index: 2.5 

element by attribute: 2.5 

series subset:
 d    1.5
f    2.5
h    3.5
dtype: float64 <class 'pandas.core.series.Series'> 



In [4]:
# Extracting elements and operations are the same as numpy array (slicing)
print(sr[:3], '\n')
print(sr[7:], '\n')
print(sr[::3], '\n')

# Fancy indexing works on Series, too
print(sr[sr > 3], '\n')

# You can also pass Series to numpy funtions
print(np.exp(sr), '\n')
print("Series mean:", np.mean(sr), ", std:", np.std(sr), '\n')

a    0.0
b    0.5
c    1.0
dtype: float64 

h    3.5
i    4.0
j    4.5
dtype: float64 

a    0.0
d    1.5
g    3.0
j    4.5
dtype: float64 

h    3.5
i    4.0
j    4.5
dtype: float64 

a     1.000000
b     1.648721
c     2.718282
d     4.481689
e     7.389056
f    12.182494
g    20.085537
h    33.115452
i    54.598150
j    90.017131
dtype: float64 

Series mean: 2.25 , std: 1.4361406616345072 



Series may contain non-omogeneous data; in this case, the data type is referred to as `object`. Non-homogeneous data is normally handeled also by Pandas and does not represent a problem, however this pays the price of less time-efficient operations.

In [5]:
# Series can be created from a python dictionary, too
# Note that the elements can be of different types
d = {'b' : 1, 'a' : 'cat', 'c' : [2, 3]}
so = pd.Series(d) # alternative constructor that taks a dict as the only input
print(so, '\n')

b         1
a       cat
c    [2, 3]
dtype: object 



A key difference between Pandas Series and Numpy arrays is that operations between Series **automatically align the data based on the label**.

Thus, you can write operations without considering whether the Series involved have the same labels, or even the same size.

If there is no matching element, the resulting value would be a `NaN`.

In [6]:
s = pd.Series(np.arange(5), index=tuple(letters[:5]))
print("series:\n", s, '\n')

s1 = s[1:]
print("shifted series:\n", s1, '\n')

s2 = s1 + s
print("shifted sum:\n", s2, '\n')

s3 = s1 + s[:-1]
print("double shifted sum:\n", s3, '\n')

series:
 a    0
b    1
c    2
d    3
e    4
dtype: int32 

shifted series:
 b    1
c    2
d    3
e    4
dtype: int32 

shifted sum:
 a    NaN
b    2.0
c    4.0
d    6.0
e    8.0
dtype: float64 

double shifted sum:
 a    NaN
b    2.0
c    4.0
d    6.0
e    NaN
dtype: float64 



### Time series

**Datetime**

When dealing with time, Python provides the `datetime` library that allows to store the date and time in an dedicated object, which possess several methods to access the relevant quantities (day, month, year, hours, minutes, seconds, ...)

In [15]:
# To define a date, the datetime module is very useful
import datetime as dt

date = dt.date.today()
print("Today's date:", date)

# specify year, month, day, hour, minutes, seconds, and microseconds
date = dt.datetime(2009, 11, 12, 10, 45, 10, 15)
print("Date and time:", date)
print("Month:", date.month, "and minutes:", date.minute)

Today's date: 2024-01-12
Date and time: 2009-11-12 10:45:10.000015
Month: 11 and minutes: 45


**Pandas Timestamps**

Timestamped data is the most basic type of time series data that associates values with points in time.

Functions like `pd.to_datetime` can be used to convert between different formats and, for instance, when reading the time stored as a string from a dataset:

In [16]:
# Get the timestamp, which is the nanoseconds from January 1st 1970 (Unix time)
tstamp = pd.Timestamp(date)
#tstamp = pd.Timestamp(dt.datetime(1970, 1, 1, 0, 0, 0, 1))
print("Timestamp:", tstamp.value)

# when creating a timestamp the format can be explicitly passed
ts = pd.to_datetime('2010/11/12', format='%Y/%m/%d')
print("Time:", ts, ", timestamp:", ts.value, ", type:", type(ts))

ts = pd.to_datetime('12-11-2010 10:39', format='%d-%m-%Y %H:%M')
print("Time:", ts, ", timestamp:", ts.value, ", type:", type(ts))

Timestamp: 1258022710000015000
Time: 2010-11-12 00:00:00 , timestamp: 1289520000000000000 , type: <class 'pandas._libs.tslibs.timestamps.Timestamp'>
Time: 2010-11-12 10:39:00 , timestamp: 1289558340000000000 , type: <class 'pandas._libs.tslibs.timestamps.Timestamp'>


**Pandas Date range**

Time series are very often used to describe the behaviour of a quantity as a function of time. Pandas has a special type of index for that, `DatetimeIndex`, that can be created e.g. with the function `pd.data_range()`.

In [18]:
# create DatetimeIndex using ranges:
days = pd.date_range(date, periods=7, freq='D')
print("7 days range:", days)

seconds = pd.date_range(date, periods=3600, freq='s')
print("1 hour in seconds:", seconds)

7 days range: DatetimeIndex(['2020-11-12 10:45:10.000015', '2020-11-13 10:45:10.000015',
               '2020-11-14 10:45:10.000015', '2020-11-15 10:45:10.000015',
               '2020-11-16 10:45:10.000015', '2020-11-17 10:45:10.000015',
               '2020-11-18 10:45:10.000015'],
              dtype='datetime64[ns]', freq='D')
1 hour in seconds: DatetimeIndex(['2020-11-12 10:45:10.000015', '2020-11-12 10:45:11.000015',
               '2020-11-12 10:45:12.000015', '2020-11-12 10:45:13.000015',
               '2020-11-12 10:45:14.000015', '2020-11-12 10:45:15.000015',
               '2020-11-12 10:45:16.000015', '2020-11-12 10:45:17.000015',
               '2020-11-12 10:45:18.000015', '2020-11-12 10:45:19.000015',
               ...
               '2020-11-12 11:45:00.000015', '2020-11-12 11:45:01.000015',
               '2020-11-12 11:45:02.000015', '2020-11-12 11:45:03.000015',
               '2020-11-12 11:45:04.000015', '2020-11-12 11:45:05.000015',
               '2020-11-12 11

To learn more about the frequency strings, please check the [documentation](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases).

A standard series can be created, and (a range of) elements can be used as indices:

In [19]:
print("index:\n", days, '\n')
tseries = pd.Series(np.random.normal(10, 1, len(days)), index=days)
print("time series:\n", days, '\n')
# Extracting elements
print("slice by position:\n", tseries[0:4], '\n')
print("slice by date range:\n", tseries['2020-9-11' : '2020-9-14'], '\n') # note that includes end time

index:
 DatetimeIndex(['2020-11-12 10:45:10.000015', '2020-11-13 10:45:10.000015',
               '2020-11-14 10:45:10.000015', '2020-11-15 10:45:10.000015',
               '2020-11-16 10:45:10.000015', '2020-11-17 10:45:10.000015',
               '2020-11-18 10:45:10.000015'],
              dtype='datetime64[ns]', freq='D') 

time series:
 DatetimeIndex(['2020-11-12 10:45:10.000015', '2020-11-13 10:45:10.000015',
               '2020-11-14 10:45:10.000015', '2020-11-15 10:45:10.000015',
               '2020-11-16 10:45:10.000015', '2020-11-17 10:45:10.000015',
               '2020-11-18 10:45:10.000015'],
              dtype='datetime64[ns]', freq='D') 

slice by position:
 2020-11-12 10:45:10.000015    10.626062
2020-11-13 10:45:10.000015    11.392994
2020-11-14 10:45:10.000015    10.390182
2020-11-15 10:45:10.000015    10.564125
Freq: D, dtype: float64 

slice by date range:
 Series([], Freq: D, dtype: float64) 



`pd.to_datetime` can also be used to create a `DatetimeIndex` if the argument is a list:

In [20]:
print(pd.to_datetime([1, 2, 3, 4], unit='D', origin=pd.Timestamp('1980-02-03')))

DatetimeIndex(['1980-02-04', '1980-02-05', '1980-02-06', '1980-02-07'], dtype='datetime64[ns]', freq=None)


## DataFrame

A pandas DataFrame can be thought as a tabular spreadsheet, although the performance, the functionalities and the capabilities are very different.

Similarly to Series, the DataFrame structure also contains labeled axes (rows and columns). Arithmetic operations **align on both row and column labels**. Each column in a DataFrame is a Series object: as a matter of fact, a DataFrame can be thought of as a dict-like container for Series objects.

The elements can be of all types, and missing data could be present too (represented as `NaN`).

For future reference (or for people already familiar with R), a pandas DataFrame is also similar to the R DataFrame.

Link to the official [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

### Constructor

A DataFrame objects can be created by passing a dictionary of objects. Note that the dictionary values are not omogeneous and do not have the same length. In these cases, pandas will automatically adjust the sizes, by replicating the content or adding NaN if necessary.

In [17]:
df = pd.DataFrame({
    'A' : 1.,
    'B' : pd.Timestamp('20130102'),
    'C' : pd.Series(3, index=range(4), dtype='float32'),
    'D' : np.arange(7, 11),
    'E' : pd.Categorical(["test", "train", "test", "train"]), # a Series that represents a category label
})
# the keys of the dictionary represent the labels of the columns
# since no index is specified, the simplest one [0, 1, 2, ...] is added by Pandas automatically

df

,A,B,C,D,E
0,1.0,2013-01-02,3.0,7,test
1,1.0,2013-01-02,3.0,8,train
2,1.0,2013-01-02,3.0,9,test
3,1.0,2013-01-02,3.0,10,train


An example of DataFrame with a `DatatimeIndex` object as index:

In [20]:
entries = 10
columns = ['A', 'B', 'C', 'D']
dates = pd.date_range('11/9/2020 14:45:00', freq='h', periods=entries) # days/month/year
df = pd.DataFrame(np.random.randn(entries, len(columns)), index=dates, columns=columns)
df # pay attention that the date is printed as year-day-month

,A,B,C,D
2020-11-09 14:45:00,-0.829803,-0.330790,0.481105,-1.165843
2020-11-09 15:45:00,0.024000,-0.059948,-0.050499,1.852915
2020-11-09 16:45:00,1.159216,-0.707576,-0.289636,-0.188046
2020-11-09 17:45:00,-0.814326,0.580188,-1.339717,0.983038
2020-11-09 18:45:00,0.681784,-1.690074,2.191693,-0.406995
2020-11-09 19:45:00,0.209201,1.088841,1.974500,-0.546747
2020-11-09 20:45:00,1.912201,0.246638,2.333881,-1.167522
2020-11-09 21:45:00,-0.178332,0.243025,-2.240918,0.060716
2020-11-09 22:45:00,0.909904,0.442423,-0.436602,-1.249362
2020-11-09 23:45:00,-2.037713,1.124931,-1.049207,-0.285753


### Viewing Data

In [21]:
df.head()

,A,B,C,D
2020-11-09 14:45:00,-0.829803,-0.330790,0.481105,-1.165843
2020-11-09 15:45:00,0.024000,-0.059948,-0.050499,1.852915
2020-11-09 16:45:00,1.159216,-0.707576,-0.289636,-0.188046
2020-11-09 17:45:00,-0.814326,0.580188,-1.339717,0.983038
2020-11-09 18:45:00,0.681784,-1.690074,2.191693,-0.406995


In [22]:
df.tail(4)

,A,B,C,D
2020-11-09 20:45:00,1.912201,0.246638,2.333881,-1.167522
2020-11-09 21:45:00,-0.178332,0.243025,-2.240918,0.060716
2020-11-09 22:45:00,0.909904,0.442423,-0.436602,-1.249362
2020-11-09 23:45:00,-2.037713,1.124931,-1.049207,-0.285753


In [23]:
df.index

DatetimeIndex(['2020-11-09 14:45:00', '2020-11-09 15:45:00',
               '2020-11-09 16:45:00', '2020-11-09 17:45:00',
               '2020-11-09 18:45:00', '2020-11-09 19:45:00',
               '2020-11-09 20:45:00', '2020-11-09 21:45:00',
               '2020-11-09 22:45:00', '2020-11-09 23:45:00'],
              dtype='datetime64[ns]', freq='H')

In [24]:
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [25]:
df.values

array([[-0.8298028 , -0.3307903 ,  0.48110505, -1.16584325],
       [ 0.02399984, -0.05994751, -0.0504985 ,  1.85291487],
       [ 1.15921553, -0.70757565, -0.2896363 , -0.18804613],
       [-0.81432637,  0.5801884 , -1.33971739,  0.98303757],
       [ 0.68178365, -1.69007371,  2.19169276, -0.40699541],
       [ 0.20920081,  1.0888408 ,  1.97449952, -0.54674688],
       [ 1.91220107,  0.2466381 ,  2.3338811 , -1.16752239],
       [-0.17833243,  0.24302516, -2.24091753,  0.06071573],
       [ 0.90990351,  0.44242333, -0.43660154, -1.24936192],
       [-2.03771257,  1.12493094, -1.04920665, -0.28575308]])

In [26]:
df.describe() #di ogni colonna

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,0.103613,0.093766,0.157460,-0.211360
std,1.143377,0.848882,1.575053,0.990029
min,-2.037713,-1.690074,-2.240918,-1.249362
25%,-0.655328,-0.263080,-0.896055,-1.011069
50%,0.116600,0.244832,-0.170067,-0.346374
75%,0.852874,0.545747,1.601151,-0.001475
max,1.912201,1.124931,2.333881,1.852915


In [27]:
df.T

,2020-11-09 14:45:00,2020-11-09 15:45:00,2020-11-09 16:45:00,2020-11-09 17:45:00,2020-11-09 18:45:00,2020-11-09 19:45:00,2020-11-09 20:45:00,2020-11-09 21:45:00,2020-11-09 22:45:00,2020-11-09 23:45:00
A,-0.829803,0.024000,1.159216,-0.814326,0.681784,0.209201,1.912201,-0.178332,0.909904,-2.037713
B,-0.330790,-0.059948,-0.707576,0.580188,-1.690074,1.088841,0.246638,0.243025,0.442423,1.124931
C,0.481105,-0.050499,-0.289636,-1.339717,2.191693,1.974500,2.333881,-2.240918,-0.436602,-1.049207
D,-1.165843,1.852915,-0.188046,0.983038,-0.406995,-0.546747,-1.167522,0.060716,-1.249362,-0.285753


In [28]:
df.sort_index(axis=1, ascending=False)

,D,C,B,A
2020-11-09 14:45:00,-1.165843,0.481105,-0.330790,-0.829803
2020-11-09 15:45:00,1.852915,-0.050499,-0.059948,0.024000
2020-11-09 16:45:00,-0.188046,-0.289636,-0.707576,1.159216
2020-11-09 17:45:00,0.983038,-1.339717,0.580188,-0.814326
2020-11-09 18:45:00,-0.406995,2.191693,-1.690074,0.681784
2020-11-09 19:45:00,-0.546747,1.974500,1.088841,0.209201
2020-11-09 20:45:00,-1.167522,2.333881,0.246638,1.912201
2020-11-09 21:45:00,0.060716,-2.240918,0.243025,-0.178332
2020-11-09 22:45:00,-1.249362,-0.436602,0.442423,0.909904
2020-11-09 23:45:00,-0.285753,-1.049207,1.124931,-2.037713


In [29]:
df.sort_values(by="C", ascending=False)

,A,B,C,D
2020-11-09 20:45:00,1.912201,0.246638,2.333881,-1.167522
2020-11-09 18:45:00,0.681784,-1.690074,2.191693,-0.406995
2020-11-09 19:45:00,0.209201,1.088841,1.974500,-0.546747
2020-11-09 14:45:00,-0.829803,-0.330790,0.481105,-1.165843
2020-11-09 15:45:00,0.024000,-0.059948,-0.050499,1.852915
2020-11-09 16:45:00,1.159216,-0.707576,-0.289636,-0.188046
2020-11-09 22:45:00,0.909904,0.442423,-0.436602,-1.249362
2020-11-09 23:45:00,-2.037713,1.124931,-1.049207,-0.285753
2020-11-09 17:45:00,-0.814326,0.580188,-1.339717,0.983038
2020-11-09 21:45:00,-0.178332,0.243025,-2.240918,0.060716


### Selection

#### Slicing

DataFrame slicing allows to select a subset of the DataFrame, or an entire column (a Series):

In [30]:
# standard and safe
print(df['A'], '\n', type(df['A']), '\n') # Returns a Series (a column)

# equivalent but dangerous (imagine blank spaces in the name of the column, or a column named "T")
print(df.A, '\n')

2020-11-09 14:45:00   -0.829803
2020-11-09 15:45:00    0.024000
2020-11-09 16:45:00    1.159216
2020-11-09 17:45:00   -0.814326
2020-11-09 18:45:00    0.681784
2020-11-09 19:45:00    0.209201
2020-11-09 20:45:00    1.912201
2020-11-09 21:45:00   -0.178332
2020-11-09 22:45:00    0.909904
2020-11-09 23:45:00   -2.037713
Freq: H, Name: A, dtype: float64 
 <class 'pandas.core.series.Series'> 

2020-11-09 14:45:00   -0.829803
2020-11-09 15:45:00    0.024000
2020-11-09 16:45:00    1.159216
2020-11-09 17:45:00   -0.814326
2020-11-09 18:45:00    0.681784
2020-11-09 19:45:00    0.209201
2020-11-09 20:45:00    1.912201
2020-11-09 21:45:00   -0.178332
2020-11-09 22:45:00    0.909904
2020-11-09 23:45:00   -2.037713
Freq: H, Name: A, dtype: float64 



Numpy-like slicing by row ranges is possible, and usually returns a **view** of the original DataFrame:

In [31]:
# selecting rows by range. Returns another DataFrame (usually a view)
print(df[0:3], '\n')

# or by index range
print(df["2020-11-09 14:45:00" : "2020-11-09 16:45:00"])

                            A         B         C         D
2020-11-09 14:45:00 -0.829803 -0.330790  0.481105 -1.165843
2020-11-09 15:45:00  0.024000 -0.059948 -0.050499  1.852915
2020-11-09 16:45:00  1.159216 -0.707576 -0.289636 -0.188046 

                            A         B         C         D
2020-11-09 14:45:00 -0.829803 -0.330790  0.481105 -1.165843
2020-11-09 15:45:00  0.024000 -0.059948 -0.050499  1.852915
2020-11-09 16:45:00  1.159216 -0.707576 -0.289636 -0.188046


#### Selection by label

The most common way to select elements, rows, or columns in a DataFrame is by using the `.loc[]` method.

`.loc` supports multi-indexing, and usually returns a **copy** of the DataFrame.

In [32]:
# getting a part of the DataFrame (in this case, a row)) using a label. Returns a Series
dfs = df.loc[dates[0]] # equivalent to df.loc[df.index[0]]
print(dfs, '\n', type(dfs), '\n')

A   -0.829803
B   -0.330790
C    0.481105
D   -1.165843
Name: 2020-11-09 14:45:00, dtype: float64 
 <class 'pandas.core.series.Series'> 



In [33]:
# selecting on a multi-axis by label:
dfa = df.loc[:, ['A','B']]
dfa

,A,B
2020-11-09 14:45:00,-0.829803,-0.330790
2020-11-09 15:45:00,0.024000,-0.059948
2020-11-09 16:45:00,1.159216,-0.707576
2020-11-09 17:45:00,-0.814326,0.580188
2020-11-09 18:45:00,0.681784,-1.690074
2020-11-09 19:45:00,0.209201,1.088841
2020-11-09 20:45:00,1.912201,0.246638
2020-11-09 21:45:00,-0.178332,0.243025
2020-11-09 22:45:00,0.909904,0.442423
2020-11-09 23:45:00,-2.037713,1.124931


In [36]:
# showing label slicing, both endpoints are included:
df.loc['2020-11-09 18:45:00':'2020-11-09 20:45:00', ['A','B']]

,A,B
2020-11-09 18:45:00,-1.210521,-0.793152
2020-11-09 19:45:00,0.158264,-1.535905
2020-11-09 20:45:00,-1.813941,-1.096709


In [34]:
# getting an individual element
print(df.loc[dates[1], 'A'], '\n', type(df.loc[dates[1], 'A']), '\n')

0.02399983524712154 
 <class 'numpy.float64'> 



The `.at()` method is equivalent to `.loc[]`. Use `at` if you only need to get or set a single value in a DataFrame or Series.

In [ ]:
print(df.at[dates[1], 'A'])

#### Selecting by position

`.iloc[]` is similar to `.loc[]`, but instead of labels, it uses pure integer-location based indexing for selection by position.

But differently from `.loc[]`, `.iloc[]` usually returns a **view**, not a copy.

In [35]:
# select via the position of the passed integers:
print(df.iloc[3], '\n', type(df.iloc[3]), '\n')

A   -0.814326
B    0.580188
C   -1.339717
D    0.983038
Name: 2020-11-09 17:45:00, dtype: float64 
 <class 'pandas.core.series.Series'> 



If you specify just one axis or idex, a Series is returned. If you specify both axis or indices, you get a DataFrame instead:

In [36]:
# row and column ranges selected with numpy-like notation:
dfv = df.iloc[3:5, 0:2]
print(dfv, '\n', type(df.iloc[3:5, 0:2]), '\n')

                            A         B
2020-11-09 17:45:00 -0.814326  0.580188
2020-11-09 18:45:00  0.681784 -1.690074 
 <class 'pandas.core.frame.DataFrame'> 



In [37]:
# selection of multiple elements with lists
df.iloc[[1, 2, 4], [0, 2]] # selecting rows 1,2 and 4 for columns 0 and 2

,A,C
2020-11-09 15:45:00,0.024000,-0.050499
2020-11-09 16:45:00,1.159216,-0.289636
2020-11-09 18:45:00,0.681784,2.191693


In [38]:
# slicing rows explicitly
df.iloc[1:3, :]

# slicing columns explicitly
df.iloc[:, 1:3]

,B,C
2020-11-09 14:45:00,-0.330790,0.481105
2020-11-09 15:45:00,-0.059948,-0.050499
2020-11-09 16:45:00,-0.707576,-0.289636
2020-11-09 17:45:00,0.580188,-1.339717
2020-11-09 18:45:00,-1.690074,2.191693
2020-11-09 19:45:00,1.088841,1.974500
2020-11-09 20:45:00,0.246638,2.333881
2020-11-09 21:45:00,0.243025,-2.240918
2020-11-09 22:45:00,0.442423,-0.436602
2020-11-09 23:45:00,1.124931,-1.049207


Similary to `.loc[]` and `.at[]`, there is also `.iat[]` alongside `.iloc[]`:

In [39]:
# selecting an individual element by position: no difference between iloc and iat
print(df.iloc[1,1], ", type:", type(df.iloc[1,1]))
print(df.iat[1,1], ", type:", type(df.iat[1,1]))

-0.0599475143693406 , type: <class 'numpy.float64'>
-0.0599475143693406 , type: <class 'numpy.float64'>


#### Masks

Boolean masks can be used in the same way as Numpy, and they represent a very powerful way of filtering out data with certain features. Just like Numpy fancy indexing, using a mask usually returns a **copy** of the DataFrame.

In [40]:
# Selecting on the basis of boolean conditions applied to the whole DataFrame
dfc = df[df > 0]
dfc.iat[0, 0] = -99
# a DataFrame with the same shape is returned, with NaN's where condition is not met
# Note that when a NaN is present in a column of integers, the column (Series) is casted to float
dfc

,A,B,C,D
2020-11-09 14:45:00,-99.000000,NaN,0.481105,NaN
2020-11-09 15:45:00,0.024000,NaN,NaN,1.852915
2020-11-09 16:45:00,1.159216,NaN,NaN,NaN
2020-11-09 17:45:00,NaN,0.580188,NaN,0.983038
2020-11-09 18:45:00,0.681784,NaN,2.191693,NaN
2020-11-09 19:45:00,0.209201,1.088841,1.974500,NaN
2020-11-09 20:45:00,1.912201,0.246638,2.333881,NaN
2020-11-09 21:45:00,NaN,0.243025,NaN,0.060716
2020-11-09 22:45:00,0.909904,0.442423,NaN,NaN
2020-11-09 23:45:00,NaN,1.124931,NaN,NaN


In [41]:
# Filter by a boolean condition on the values of a single column
mask = dfc['B'] < 0.5
mask

2020-11-09 14:45:00    False
2020-11-09 15:45:00    False
2020-11-09 16:45:00    False
2020-11-09 17:45:00    False
2020-11-09 18:45:00    False
2020-11-09 19:45:00    False
2020-11-09 20:45:00     True
2020-11-09 21:45:00     True
2020-11-09 22:45:00     True
2020-11-09 23:45:00    False
Freq: H, Name: B, dtype: bool

In [42]:
# Filter only the rows that correspond to a True in the Series used as mask
dfc[mask]

,A,B,C,D
2020-11-09 20:45:00,1.912201,0.246638,2.333881,NaN
2020-11-09 21:45:00,NaN,0.243025,NaN,0.060716
2020-11-09 22:45:00,0.909904,0.442423,NaN,NaN


**Queries**

Pandas uses a database-like engine to select elements according to a query on the columns of the DataFrame:

In [38]:
dfq = df.query('C > 0.5')
dfq

,A,B,C,D
2020-11-09 15:45:00,0.793111,0.662657,2.037128,1.141570
2020-11-09 16:45:00,1.517130,2.441813,0.972791,0.840373
2020-11-09 18:45:00,-1.210521,-0.793152,0.785296,-0.742990
2020-11-09 20:45:00,-1.813941,-1.096709,2.065599,0.615972
2020-11-09 21:45:00,-0.509033,2.246361,0.895052,-1.696757
2020-11-09 23:45:00,0.172553,-0.390135,1.216394,0.138420


which is equivalent to `dfq = df[df['C'] > 0.5]`:

In [ ]:
dfw = df[df['C'] > 0.5]
dfw

### Copy and views in DataFrames

The view/copy behaviour in Pandas is not as easy as it may appear, as there are counter-intuitive exceptions. There was a plan to fix this by quite some time, but a fix has not been deployed yet.

Check this discussion [here](https://www.practicaldatascience.org/html/views_and_copies_in_pandas.html):

    In numpy, the rules for when you get views and when you don’t are a little complicated, but they are consistent: certain behaviors (like simple indexing) will always return a view, and others (fancy indexing) will never return a view.

    But in pandas, whether you get a view or not—and whether changes made to a view will propagate back to the original DataFrame—depends on the structure and data types in the original DataFrame.


In summary, there is only one way to write safe code when dealing with slides of a dataframe: after every instruction that selects a subset of a DataFrame, force the copy by appending `.copy()` to the slice.

### Assignement

Assignment is typically performed after a selection:

In [43]:
# Make sure to copy the DataFrame if you plan to modify it, and you don't want to change the original object
dfa = df.copy()

# setting values by label (same as by position)
dfa.at[dates[0], 'A'] = -1

# setting and assigning a numpy array
dfa['D'] = np.array([5] * len(dfa))

# defining a brand new column by means of a pd.Series: indexes must be the same!
dfa['E'] = pd.Series(np.arange(len(dfa))*2, index=dfa.index)

# using masks for assigment
dfa[dfa < 0] = -dfa

dfa

,A,B,C,D,E
2020-11-09 14:45:00,1.000000,0.330790,0.481105,5,0
2020-11-09 15:45:00,0.024000,0.059948,0.050499,5,2
2020-11-09 16:45:00,1.159216,0.707576,0.289636,5,4
2020-11-09 17:45:00,0.814326,0.580188,1.339717,5,6
2020-11-09 18:45:00,0.681784,1.690074,2.191693,5,8
2020-11-09 19:45:00,0.209201,1.088841,1.974500,5,10
2020-11-09 20:45:00,1.912201,0.246638,2.333881,5,12
2020-11-09 21:45:00,0.178332,0.243025,2.240918,5,14
2020-11-09 22:45:00,0.909904,0.442423,0.436602,5,16
2020-11-09 23:45:00,2.037713,1.124931,1.049207,5,18


### Dropping

Dropping columns is an example of **a method that does not modify the original object**, and returns a new modified object. In other words, if you want to keep the modified DataFrame, perform a new assignment:

```python
df = df.drop(...)
```
Alternatively, the modification of the original object can be forced by specifying `inplace=True` among the arguments.

In [45]:
dfb = dfa.copy()

# Dropping by column..
dfb.drop(['E'], axis=1)
dfb

,A,B,C,D,E
2020-11-09 14:45:00,1.000000,0.330790,0.481105,5,0
2020-11-09 15:45:00,0.024000,0.059948,0.050499,5,2
2020-11-09 16:45:00,1.159216,0.707576,0.289636,5,4
2020-11-09 17:45:00,0.814326,0.580188,1.339717,5,6
2020-11-09 18:45:00,0.681784,1.690074,2.191693,5,8
2020-11-09 19:45:00,0.209201,1.088841,1.974500,5,10
2020-11-09 20:45:00,1.912201,0.246638,2.333881,5,12
2020-11-09 21:45:00,0.178332,0.243025,2.240918,5,14
2020-11-09 22:45:00,0.909904,0.442423,0.436602,5,16
2020-11-09 23:45:00,2.037713,1.124931,1.049207,5,18


As you can see, there is no effect on the original object. That's because  new object is returned instead. To keep it, there are two alternatives:

In [46]:
dfc = dfb.drop(columns=['E'])
dfc

,A,B,C,D
2020-11-09 14:45:00,1.000000,0.330790,0.481105,5
2020-11-09 15:45:00,0.024000,0.059948,0.050499,5
2020-11-09 16:45:00,1.159216,0.707576,0.289636,5
2020-11-09 17:45:00,0.814326,0.580188,1.339717,5
2020-11-09 18:45:00,0.681784,1.690074,2.191693,5
2020-11-09 19:45:00,0.209201,1.088841,1.974500,5
2020-11-09 20:45:00,1.912201,0.246638,2.333881,5
2020-11-09 21:45:00,0.178332,0.243025,2.240918,5
2020-11-09 22:45:00,0.909904,0.442423,0.436602,5
2020-11-09 23:45:00,2.037713,1.124931,1.049207,5


In [42]:
dfb.drop(columns=['E'], inplace=True) # equivalent to the previous one, but the original object has been replace inplace
dfb

,A,B,C,D
2020-11-09 14:45:00,1.000000,1.621450,2.534742,5
2020-11-09 15:45:00,0.793111,0.662657,2.037128,5
2020-11-09 16:45:00,1.517130,2.441813,0.972791,5
2020-11-09 17:45:00,0.839773,1.538780,0.391025,5
2020-11-09 18:45:00,1.210521,0.793152,0.785296,5
2020-11-09 19:45:00,0.158264,1.535905,0.703959,5
2020-11-09 20:45:00,1.813941,1.096709,2.065599,5
2020-11-09 21:45:00,0.509033,2.246361,0.895052,5
2020-11-09 22:45:00,0.812058,0.557555,0.017264,5
2020-11-09 23:45:00,0.172553,0.390135,1.216394,5


### Dealing with missing data

Pandas primarily uses the value `np.nan` to represent missing data. It is by default not included in computations. If there is a `NaN` entry in a Series of integers, the type of the Series will be changed to floats.

In [43]:
df_wNan = dfb[dfb > 0.5]
df_wNan

,A,B,C,D
2020-11-09 14:45:00,1.000000,1.621450,2.534742,5
2020-11-09 15:45:00,0.793111,0.662657,2.037128,5
2020-11-09 16:45:00,1.517130,2.441813,0.972791,5
2020-11-09 17:45:00,0.839773,1.538780,NaN,5
2020-11-09 18:45:00,1.210521,0.793152,0.785296,5
2020-11-09 19:45:00,NaN,1.535905,0.703959,5
2020-11-09 20:45:00,1.813941,1.096709,2.065599,5
2020-11-09 21:45:00,0.509033,2.246361,0.895052,5
2020-11-09 22:45:00,0.812058,0.557555,NaN,5
2020-11-09 23:45:00,NaN,NaN,1.216394,5


In [47]:
# dropping rows with at least a Nan
c = df_wNan.dropna(how='any')
c

,A,B,C,D
2020-11-09 14:45:00,1.000000,1.621450,2.534742,5
2020-11-09 15:45:00,0.793111,0.662657,2.037128,5
2020-11-09 16:45:00,1.517130,2.441813,0.972791,5
2020-11-09 18:45:00,1.210521,0.793152,0.785296,5
2020-11-09 20:45:00,1.813941,1.096709,2.065599,5
2020-11-09 21:45:00,0.509033,2.246361,0.895052,5


In [45]:
# getting a mask
df_wNan.isna()
# df_wNan.notna()

,A,B,C,D
2020-11-09 14:45:00,False,False,False,False
2020-11-09 15:45:00,False,False,False,False
2020-11-09 16:45:00,False,False,False,False
2020-11-09 17:45:00,False,False,True,False
2020-11-09 18:45:00,False,False,False,False
2020-11-09 19:45:00,True,False,False,False
2020-11-09 20:45:00,False,False,False,False
2020-11-09 21:45:00,False,False,False,False
2020-11-09 22:45:00,False,False,True,False
2020-11-09 23:45:00,True,True,False,False


In [46]:
# filling missing data (not recommended, unless you really mean it)
df_wNan.fillna(value=0)

,A,B,C,D
2020-11-09 14:45:00,1.000000,1.621450,2.534742,5
2020-11-09 15:45:00,0.793111,0.662657,2.037128,5
2020-11-09 16:45:00,1.517130,2.441813,0.972791,5
2020-11-09 17:45:00,0.839773,1.538780,0.000000,5
2020-11-09 18:45:00,1.210521,0.793152,0.785296,5
2020-11-09 19:45:00,0.000000,1.535905,0.703959,5
2020-11-09 20:45:00,1.813941,1.096709,2.065599,5
2020-11-09 21:45:00,0.509033,2.246361,0.895052,5
2020-11-09 22:45:00,0.812058,0.557555,0.000000,5
2020-11-09 23:45:00,0.000000,0.000000,1.216394,5


### Operations

Operations on the elements of a DataFrame are quite straightforward, as the syntax is the same as the one used for Series. Also for DataFrames, operations are performed between elements that share the same labels. Operations on columns are extremly fast, almost as fast as the actual operation between elements in a row.

In [48]:
# Some statistics (mean() just as an example)
# on rows
print(df.mean(axis=0), '\n')
# on columns
print(df.mean(axis=1), '\n')

A   -0.362203
B    0.200981
C    0.432527
D    0.299207
dtype: float64 

2020-11-09 14:45:00   -1.517309
2020-11-09 15:45:00    1.158617
2020-11-09 16:45:00    1.443026
2020-11-09 17:45:00    0.191823
2020-11-09 18:45:00   -0.490342
2020-11-09 19:45:00   -0.284625
2020-11-09 20:45:00   -0.057270
2020-11-09 21:45:00    0.233906
2020-11-09 22:45:00    0.464148
2020-11-09 23:45:00    0.284308
Freq: H, dtype: float64 



In [47]:
# Global operations on columns
df.apply(np.sum) # or whatever function defined by the user

A    1.036130
B    0.937660
C    1.574601
D   -2.113601
dtype: float64

In [48]:
# Also lambda functions are accepted
df.apply(lambda x: x - x.max())

,A,B,C,D
2020-11-09 14:45:00,-2.742004,-1.455721,-1.852776,-3.018758
2020-11-09 15:45:00,-1.888201,-1.184878,-2.384380,0.000000
2020-11-09 16:45:00,-0.752986,-1.832507,-2.623517,-2.040961
2020-11-09 17:45:00,-2.726527,-0.544743,-3.673598,-0.869877
2020-11-09 18:45:00,-1.230417,-2.815005,-0.142188,-2.259910
2020-11-09 19:45:00,-1.703000,-0.036090,-0.359382,-2.399662
2020-11-09 20:45:00,0.000000,-0.878293,0.000000,-3.020437
2020-11-09 21:45:00,-2.090533,-0.881906,-4.574799,-1.792199
2020-11-09 22:45:00,-1.002298,-0.682508,-2.770483,-3.102277
2020-11-09 23:45:00,-3.949914,0.000000,-3.383088,-2.138668


In [49]:
# syntax is as usual similar to that of numpy arrays
df['S'] = df['A'] + df['C']
df

,A,B,C,D,S
2020-11-09 14:45:00,-0.829803,-0.330790,0.481105,-1.165843,-0.348698
2020-11-09 15:45:00,0.024000,-0.059948,-0.050499,1.852915,-0.026499
2020-11-09 16:45:00,1.159216,-0.707576,-0.289636,-0.188046,0.869579
2020-11-09 17:45:00,-0.814326,0.580188,-1.339717,0.983038,-2.154044
2020-11-09 18:45:00,0.681784,-1.690074,2.191693,-0.406995,2.873476
2020-11-09 19:45:00,0.209201,1.088841,1.974500,-0.546747,2.183700
2020-11-09 20:45:00,1.912201,0.246638,2.333881,-1.167522,4.246082
2020-11-09 21:45:00,-0.178332,0.243025,-2.240918,0.060716,-2.419250
2020-11-09 22:45:00,0.909904,0.442423,-0.436602,-1.249362,0.473302
2020-11-09 23:45:00,-2.037713,1.124931,-1.049207,-0.285753,-3.086919


### Application of a function: apply vs transform

User-defined or standard functions can be applied on entire DataFrames or columns, with very short execution times.

There are two main methods, `apply()` and `transform()`:

In [50]:
def dcos(theta):
    theta = theta * (np.pi / 180)
    return np.cos(theta)

# Apply method with custom function
dfa['cosine'] = dfa["E"].apply(dcos)

# Transform method with lambda function
dfa['EplusOne'] = dfa["E"].transform(lambda x: x + 1)
dfa

,A,B,C,D,E,cosine,EplusOne
2020-11-09 14:45:00,1.000000,1.621450,2.534742,5,0,1.000000,1
2020-11-09 15:45:00,0.793111,0.662657,2.037128,5,2,0.999391,3
2020-11-09 16:45:00,1.517130,2.441813,0.972791,5,4,0.997564,5
2020-11-09 17:45:00,0.839773,1.538780,0.391025,5,6,0.994522,7
2020-11-09 18:45:00,1.210521,0.793152,0.785296,5,8,0.990268,9
2020-11-09 19:45:00,0.158264,1.535905,0.703959,5,10,0.984808,11
2020-11-09 20:45:00,1.813941,1.096709,2.065599,5,12,0.978148,13
2020-11-09 21:45:00,0.509033,2.246361,0.895052,5,14,0.970296,15
2020-11-09 22:45:00,0.812058,0.557555,0.017264,5,16,0.961262,17
2020-11-09 23:45:00,0.172553,0.390135,1.216394,5,18,0.951057,19


The major differences between `apply` and `transform` are:

   - Input: `apply` passes all the columns to the custom function, while `transform` passes each column.
   - Output: the custom function passed to `apply` can return a scalar, or a Series or DataFrame, while the custom function passed to `transform` must return a sequence (a Series, array or list) with the same length.

In summary, `transform` works on just one Series, and `apply` works on the entire DataFrame.

### Merge

Pandas provides various functions for easily combining together Series and DataFrames in join / merge-type operations.

**Concat**

Concatenation (adding rows) is straightforward:

In [50]:
rdf = pd.DataFrame(np.arange(40).reshape(10, 4))
rdf

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23
6,24,25,26,27
7,28,29,30,31
8,32,33,34,35
9,36,37,38,39


In [51]:
# split DataFrame into 3 pieces, row-wise
pieces = [rdf[:3], rdf[3:7], rdf[7:]]
print(pieces, '\n')
pieces[2]

[   0  1   2   3
0  0  1   2   3
1  4  5   6   7
2  8  9  10  11,     0   1   2   3
3  12  13  14  15
4  16  17  18  19
5  20  21  22  23
6  24  25  26  27,     0   1   2   3
7  28  29  30  31
8  32  33  34  35
9  36  37  38  39] 



,0,1,2,3
7,28,29,30,31
8,32,33,34,35
9,36,37,38,39


In [52]:
# put it back together
pd.concat(pieces)

# in this case, indices are already set; if they are not, indices can be ignored
#pd.concat(pieces, ignore_index=True)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23
6,24,25,26,27
7,28,29,30,31
8,32,33,34,35
9,36,37,38,39


In case of dimension mismatch, `NaN` are added where needed.

**Merge/Join**

SQL-like operations on table can be performed on DataFrames. This is a quite advanced use case, refer to the [doc](https://pandas.pydata.org/pandas-docs/stable/merging.html#merging) for more info/examples.

In [53]:
left = pd.DataFrame({'key': ['foo', 'bar'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'bar'], 'rval': [4, 5]})

pd.merge(left, right, on="key")

,key,lval,rval
0,foo,1,4
1,bar,2,5


### Grouping

In real world applications, it's quite common that several entries (row) belong to a certain entity, or "group". DataFrames have a powerful tool to perform operations on entries of the same group. The method is called `.groupby()`, and it usually involves one or more of the following steps:

* Splitting the data into groups based on some criteria
* Applying a function to each group independently
* Combining the results into a data structure


In [56]:
gdf = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B' : [1, 1, 2, 3, 2, 2, 1, 3],
                    'C' : np.arange(8),
                    'D' : np.linspace(10, -10, 8)})
gdf

,A,B,C,D
0,foo,1,0,10.000000
1,bar,1,1,7.142857
2,foo,2,2,4.285714
3,bar,3,3,1.428571
4,foo,2,4,-1.428571
5,bar,2,5,-4.285714
6,foo,1,6,-7.142857
7,foo,3,7,-10.000000


In [57]:
# Grouping and then applying the sum() 
# function to the resulting groups (effective only where numerical values are present)
gdf.groupby('A').sum()

,B,C,D
A,,,
bar,6,9,4.285714
foo,9,19,-4.285714


In [58]:
# Example: find maximum value in column D for each group, and assign the value to a new column

gdf['M'] =gdf.groupby('A')['D'].transform(np.max)
gdf

,A,B,C,D,M
0,foo,1,0,10.000000,10.000000
1,bar,1,1,7.142857,7.142857
2,foo,2,2,4.285714,10.000000
3,bar,3,3,1.428571,7.142857
4,foo,2,4,-1.428571,10.000000
5,bar,2,5,-4.285714,7.142857
6,foo,1,6,-7.142857,10.000000
7,foo,3,7,-10.000000,10.000000


### Multi-indexing

Hierarchical / Multi-level indexing allows sophisticated data analysis on higher dimensional data. In practice, it enables you to store and manipulate data with an arbitrary number of dimensions in lower dimensional data structures like Series (1D) and DataFrames (2D).

In [ ]:
# Creat multi-dimensional index
tuples = list(zip(['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
print(multi_index, '\n', type(multi_index), '\n')

# Create multi-indexed dataframe or series
s = pd.Series(np.arange(8)/np.pi, index=multi_index)
s

In [ ]:
# multi-indexing enables further features of the groupby method,
# e.g. when group-by by multiple columns
gdf.groupby(['A', 'B']).sum()

## Summary: a demonstration of the efficiency of the DataFrame

Let's go the hard way and load a (relatively) large dataset with approximately 1 million rows:

In [68]:
# Uncomment to download the file. Run the command just once
!curl https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

<a href="/s/raw/xvjzaxzz3ysphme/data_000637.txt">Found</a>.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    61    0    61    0     0    231      0 --:--:-- --:--:-- --:--:--   232


In [60]:

#file_name = !curl https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P
data = pd.read_csv("data_000637.txt")
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14
1310716,1,1,4,3869211171,763,11
1310717,1,0,64,3869211171,764,0
1310718,1,0,139,3869211171,769,0


Let's now do some operations among (elements of) columns

In [61]:
itime = dt.datetime.now()
print("Begin time:", itime)

# the one-liner command
data['WEIGHTEDSUM'] = data['TDC_CHANNEL'] * 2.1 + data['BX_COUNTER'] * 0.1 + 2

ftime = dt.datetime.now()
print("End time:", ftime)
print("Elapsed time:", ftime - itime)

data

Begin time: 2024-01-12 16:29:14.606561
End time: 2024-01-12 16:29:14.681790
Elapsed time: 0:00:00.075229


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,WEIGHTEDSUM
0,1,0,123,3869200167,2374,26,497.7
1,1,0,124,3869200167,2374,27,499.8
2,1,0,63,3869200167,2553,28,389.6
3,1,0,64,3869200167,2558,19,392.2
4,1,0,64,3869200167,2760,25,412.4
...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,208.4
1310716,1,1,4,3869211171,763,11,86.7
1310717,1,0,64,3869211171,764,0,212.8
1310718,1,0,139,3869211171,769,0,370.8


In [62]:
# the loop
def conversion(data):
    result = []
    for i in range(len(data)): 
        result.append(data.loc[data.index[i], 'TDC_CHANNEL'] * 2.1 + data.loc[data.index[i], 'BX_COUNTER'] * 0.1 + 2)
    return result

itime = dt.datetime.now()
print("Begin time:", itime)
data['WEIGHTEDSUM'] = conversion(data)
ftime = dt.datetime.now()
print("End time:", ftime)
print("Elapsed time:", ftime - itime)

data

Begin time: 2024-01-12 16:30:19.214173
End time: 2024-01-12 16:31:23.901881
Elapsed time: 0:01:04.687708


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,WEIGHTEDSUM
0,1,0,123,3869200167,2374,26,497.7
1,1,0,124,3869200167,2374,27,499.8
2,1,0,63,3869200167,2553,28,389.6
3,1,0,64,3869200167,2558,19,392.2
4,1,0,64,3869200167,2760,25,412.4
...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,208.4
1310716,1,1,4,3869211171,763,11,86.7
1310717,1,0,64,3869211171,764,0,212.8
1310718,1,0,139,3869211171,769,0,370.8
